## **SpDB Manual之系列5：数据绑定**

IMAS将tokamak的装置藐视和物理概念进行了统一的分类、分层次管理，清晰描述了物理量之间的内在约束关系，形成了树状、层次化的、静态的IMAS本体。
数据绑定的目的，是将树状结构的IMAS本体的静态的表述转化为动态的实现。通过代码实现，将一系列静态数据，或者静态的物理量绑定在动态的实现类中，让数据赋予了行为，这样的话每个数据的变化会带动与之相关的苏剧的变化。区别于传统的集成建模，传统的集成建模是将数据集成在一起，但是数据之间的关系是静态的，不会随着数据的变化而变化。而我们的方法是将数据和行为绑定在一起，赋予行为的数据是可操作的，同时带动与之关联的其他数据的变化。

In [40]:
from spdm.data.sp_property import sp_tree, sp_property, SpTree
from spdm.data.HTree import HTree, List, Dict
from spdm.data.Entry import open_entry
from spdm.data.AoS import AoS
import typing
import pprint

__getitem__和__getattr__都是Python中的特殊方法，用于访问对象的属性。它们的区别在于：

__getitem__用于访问对象的索引，例如obj[key]，其中key是索引值。
__getattr__用于访问对象的属性，例如obj.attr，其中attr是属性名。
如果对象同时实现了__getitem__和__getattr__，那么在访问对象的属性时，Python会先调用__getattr__方法，如果该方法抛出AttributeError异常，则会调用__getitem__方法。

In [14]:
### Python中的内置函数，用来返回、修改、删除一个对象的属性
from typing import Any


class Foo:
    def __getitem__(self, key: str) -> typing.Any:
        print(f"__getitem__({key})")

    def __setitem__(self, key: str, value: typing.Any):
        print(f"__setitem__({key}, {value})")

    def __delitem__(self, key: str):
        print(f"__delitem__({key})")

    def __getattr__(self, __name: str) -> Any:
        print(f"__getattr__({__name})")

    def __setattr__(self, __name: str,value) -> Any:
        print(f"__setattr__({__name})")
        
    def __delattr__(self, __name: str) -> Any:
        print(f"__delattr__({__name})")
        

In [15]:
foo=Foo()

foo["a"] 

__getitem__(a)


In [11]:
foo["a"]=1

__setitem__(a, 1)


In [8]:
del foo["a"]

__delitem__(a)


In [12]:
foo.a

__getattr__(a)


In [16]:
foo.a=5

__setattr__(a)


In [17]:
del foo.a

__delattr__(a)


In [63]:
@sp_tree
class House:
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        pprint.pprint((args, kwargs))

    level: int = 4
    length: float
    width: float

    @sp_property(units="m^2")
    def area(self) -> float:
        return self.width * self.length


@sp_tree
class Addrees:
    street: str
    city: str
    state: str
    zip: int
    building: str = "#12345"
    neighbour: str
    house: House = sp_property(label="big house", default_value={"level": 2, "area": 1000})


@sp_tree
class Data:
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

    name: str
    age: int
    hobbies: List[str]
    address: AoS[Addrees]


data = Data(
    {
        "name": "Alice",
        "age": 25,
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
                "house": {"length": 5, "width": 10},
            },
            {"street": "789 Elm St", "city": "Somewhere", "state": "CO", "zip": "24680"},
        ],
    },
    # _entry=open_entry(....)
)

data.name

'Alice'

In [53]:
data.age

25

In [25]:
data.hobbies[0]

'reading'

In [31]:
data.address[0].zip

12345

In [33]:
data.address[0].neighbour

<tags.not_found: 0>

In [58]:
house=data.address[1].house

(({'length': 5, 'width': 10},),
 {'_entry': None,
  '_parent': <__main__.Addrees object at 0x7f3be9446b30>,
  'default_value': {'area': 1000, 'level': 2},
  'label': 'big house',
  'name': 'house'})


In [59]:
house.level

2

In [60]:
house.length

5.0

In [61]:
house.width

10.0

In [62]:
house.area

50.0

In [64]:
House.area.metadata.get("units")

'm^2'

In [67]:
data.get("address/3/street", "I don't know")

"I don't know"

In [68]:
for address in data.address:
    print(address.street)

123 Main St
456 Oak St
789 Elm St
I don't know


In [71]:
data.address[0].street = "456 Main St"

In [72]:
data.address[0].street

'456 Main St'

In [ ]:
data2 = Data({}, _entry=open_entry("file://./data/address.json"))

In [ ]:

file_entry可以给data，这个明天试下。
file_entry = open_entry(f"file+geqdsk://{workdir}//data/g900003.00230_ITER_15MA_eqdsk16HR.txt/#equilibrium")

In [2]:



class Foo(SpTree):
    boo: Data = sp_property(label="boo", default_value=1.0)


class Bar(Foo):
    boo: Data = sp_property(label="boo", units="m")


@sp_tree
class Bar2(Foo):
    boo: Data = 2.1234


a = Bar()
b = Bar2()
print(a.boo._metadata)
print(b.boo._metadata)
print(b.get("boo")._metadata)

{'label': 'boo', 'name': 'boo', 'units': 'm'}
{'label': 'boo', 'name': 'boo'}
{'label': 'boo', 'name': 'boo'}
